* [Coronavirus Brasil](https://covid.saude.gov.br/)
* [Boletins Epidemiológicos - SP](http://www.saude.sp.gov.br/cve-centro-de-vigilancia-epidemiologica-prof.-alexandre-vranjac/areas-de-vigilancia/doencas-de-transmissao-respiratoria/coronavirus-covid-19/situacao-epidemiologica)

In [50]:
import requests
from requests import ReadTimeout, ConnectTimeout, HTTPError, Timeout, ConnectionError
from bs4 import BeautifulSoup
import re
import pandas as pd
import os

In [26]:
BASE_URL = 'http://www.saude.sp.gov.br'
URL_BOLETIM = 'http://www.saude.sp.gov.br/cve-centro-de-vigilancia-epidemiologica-prof.\
-alexandre-vranjac/areas-de-vigilancia/doencas-de-transmissao-respiratoria/coronavirus-covid-19/\
situacao-epidemiologica'

In [14]:
r = requests.get(URL_BOLETIM)
soup = BeautifulSoup(r.content, "html.parser")

In [15]:
soup


<!DOCTYPE HTML>

<html>
<head>
<title>Situação Epidemiológica - Secretaria da Saúde - Governo do Estado de São Paulo</title>
<meta charset="utf-8"/>
<meta content="" name="keywords"/>
<meta content="" name="description">
<meta content="width=device-width, initial-scale=1.0" name="viewport">
<link href="/resources/ses/favicon.png" rel="icon" type="image/png"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<link href="/mmp/ses/css/styles-min.css" media="screen, projection" rel="stylesheet" type="text/css"/>
<script src="/dwr-view/interface/coreService.js" type="text/javascript"></script>
<script src="/dwr-view/engine.js" type="text/javascript"></script>
<script src="/mmp/ses/js/scripts-min.js" type="text/javascript"></script>
<link href="/mmp/ses/css/print.css" media="print" rel="stylesheet" type="text/css"/>
<link href="/mmp/ses/css/ui-lightness/jquery-ui-1.8.8.custom.css" media="screen, projection" rel="stylesheet" type="text/css"/>
<!--[if IE 7]>
	<link href="/mmp/ses/css/ie

In [ ]:
<a href="/resources/cve-centro-de-vigilancia-epidemiologica/areas-de-vigilancia/doencas-de-transmissao-respiratoria/coronavirus/coronavirus110420_45situacao_epidemiologica.pdf">nº 45 - 11/04/20</a>

In [10]:
url_test = "http://www.saude.sp.gov.br" + "/resources/cve-centro-de-vigilancia-epidemiologica/areas-de-vigilancia/doencas-de-transmissao-respiratoria/coronavirus/coronavirus110420_45situacao_epidemiologica.pdf"

In [11]:
url_test

'http://www.saude.sp.gov.br/resources/cve-centro-de-vigilancia-epidemiologica/areas-de-vigilancia/doencas-de-transmissao-respiratoria/coronavirus/coronavirus110420_45situacao_epidemiologica.pdf'

In [69]:
results = soup.find_all("a", href=re.compile(".situacao_epidemiologica.pdf"), class_=None)
boletins_urls = []
for result in results:
    pdf_url = BASE_URL + result["href"]
    boletim_number, date = result.text.replace("nº", "").replace(" ", "").split("-")
    boletins_urls.append((date, boletim_number, pdf_url))
    
df = pd.DataFrame(data=boletins_urls, columns=["data", "boletim", "url"])
df.head()

,data,boletim,url
0,11/04/20,45,http://www.saude.sp.gov.br/resources/cve-centr...
1,10/04/20,44,http://www.saude.sp.gov.br/resources/cve-centr...
2,09/04/20,43,http://www.saude.sp.gov.br/resources/cve-centr...
3,08/04/20,42,http://www.saude.sp.gov.br/resources/cve-centr...
4,07/04/20,41,http://www.saude.sp.gov.br/resources/cve-centr...


In [70]:
df.tail()

,data,boletim,url
37,12/02/20,6,http://www.saude.sp.gov.br/resources/cve-centr...
38,11/02/20,5,http://www.saude.sp.gov.br/resources/cve-centr...
39,07/02/20,4,http://www.saude.sp.gov.br/resources/cve-centr...
40,06/02/20,3,http://www.saude.sp.gov.br/resources/cve-centr...
41,05/02/20,2,http://www.saude.sp.gov.br/resources/cve-centr...


In [71]:
df.shape

(42, 3)

In [47]:
directory = "./boletins"
if not os.path.exists(directory):
    os.makedirs(directory)

In [77]:
def saveBoletins(df, directory):
    not_saved = []
    for _, row in df.iterrows():
        file_path = directory + "/boletim_" + str(row.boletim) + "_" + str(row.data).replace("/", "-") + ".pdf"
        if os.path.isfile(file_path): # se arquivo já existe vá pro próximo boletim
            continue
        try:
            response = requests.get(row.url, timeout=10.0)
            with open(file_path, 'wb') as f:
                f.write(response.content)
        except (ConnectTimeout, HTTPError, ReadTimeout, Timeout, ConnectionError):
            not_saved.append(row)
            continue
            
    return pd.DataFrame(not_saved)

In [78]:
# condition = True

not_saved = saveBoletins(df, directory)

In [68]:
not_saved

,data,boletim,url
7,03/04/20,37,http://www.saude.sp.gov.brhttp://www.saude.sp....
18,19/03/20,26,http://www.saude.sp.gov.brhttp://www.saude.sp....
19,17/03/20,25,http://www.saude.sp.gov.brhttp://www.saude.sp....


In [65]:
not not_saved.empty

True